In [6]:
import cv2
from keras.models import load_model
import numpy as np
import os

In [7]:
model = load_model('/home/awis/Desktop/ML/emojinator.h5')

In [8]:
def get_emojis():
    emojis_folder = '/home/awis/Desktop/hand_emo/'
    emojis = []
    for emoji in range(len(os.listdir(emojis_folder))):
        print(emoji)
        emojis.append(cv2.imread(emojis_folder+str(emoji)+'.png', -1))
    return emojis

In [9]:
def keras_predict(model, image):
    processed = keras_process_image(image)
    pred_probab = model.predict(processed)[0]
    pred_class = list(pred_probab).index(max(pred_probab))
    return max(pred_probab), pred_class


def keras_process_image(img):
    image_x = 50
    image_y = 50
    img = cv2.resize(img, (image_x, image_y))
    img = np.array(img, dtype=np.float32)
    img = np.reshape(img, (-1, image_x, image_y, 1))
    return img

In [10]:
def overlay(image, emoji, x,y,w,h):
    emoji = cv2.resize(emoji, (w, h))
    try:
        image[y:y+h, x:x+w] = blend_transparent(image[y:y+h, x:x+w], emoji)
    except:
        pass
    return image

def blend_transparent(face_img, overlay_t_img):
    # Split out the transparency mask from the colour info
    overlay_img = overlay_t_img[:,:,:3] # Grab the BRG planes
    overlay_mask = overlay_t_img[:,:,3:]  # And the alpha plane

    # Again calculate the inverse mask
    background_mask = 255 - overlay_mask

    # Turn the masks into three channel, so we can use them as weights
    overlay_mask = cv2.cvtColor(overlay_mask, cv2.COLOR_GRAY2BGR)
    background_mask = cv2.cvtColor(background_mask, cv2.COLOR_GRAY2BGR)

    # Create a masked out face image, and masked out overlay
    # We convert the images to floating point in range 0.0 - 1.0
    face_part = (face_img * (1 / 255.0)) * (background_mask * (1 / 255.0))
    overlay_part = (overlay_img * (1 / 255.0)) * (overlay_mask * (1 / 255.0))

    # And finally just add them together, and rescale it back to an 8bit integer image
    return np.uint8(cv2.addWeighted(face_part, 255.0, overlay_part, 255.0, 0.0))

In [ ]:
emojis = get_emojis()
cap = cv2.VideoCapture(0)
x, y, w, h = 300, 50, 350, 350

while (cap.isOpened()):
        ret, img = cap.read()
        img = cv2.flip(img, 1)
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask2 = cv2.inRange(hsv, np.array([2, 50, 60]), np.array([25, 150, 255]))
        res = cv2.bitwise_and(img, img, mask=mask2)
        gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
        median = cv2.GaussianBlur(gray, (5, 5), 0)

        kernel_square = np.ones((5, 5), np.uint8)
        dilation = cv2.dilate(median, kernel_square, iterations=2)
        opening = cv2.morphologyEx(dilation, cv2.MORPH_CLOSE, kernel_square)
        ret, thresh = cv2.threshold(opening, 30, 255, cv2.THRESH_BINARY)

        thresh = thresh[y:y + h, x:x + w]
        contours = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[1]
        if len(contours) > 0:
            contour = max(contours, key=cv2.contourArea)
            if cv2.contourArea(contour) > 2500:
                x, y, w1, h1 = cv2.boundingRect(contour)
                newImage = thresh[y:y + h1, x:x + w1]
                newImage = cv2.resize(newImage, (50, 50))
                pred_probab, pred_class = keras_predict(model, newImage)
                print(pred_class, pred_probab)
                img = overlay(img, emojis[pred_class], 400, 250, 90, 90)

        x, y, w, h = 300, 50, 350, 350
        cv2.imshow("Frame", img)
        cv2.imshow("Contours", thresh)
        k = cv2.waitKey(10)
        if k == 27:
            break

0
1
2
3
4
5
6
7
8
9
10
11
12
1 0.99997365
1 0.99999213
4 0.70792115
7 0.9973972
7 0.9848369
7 0.5176324
9 0.9579279
9 0.9992023
1 0.9849237
1 0.9996611
1 0.96811724
1 0.72413343
9 0.91332483
1 0.8826909
9 0.7888199
1 0.99962926
1 0.9998504
1 0.5247776
1 0.6926604
9 0.9255674
9 0.83256423
1 0.99988174
1 0.6634013
1 0.99824023
1 0.99989665
9 0.6670464
1 0.99974316
9 0.7462437
1 0.9205323
1 0.99934834
1 0.9455297
1 0.8117726
11 0.71153146
11 0.54775506
11 0.5492419
11 0.8148644
8 0.6520702
11 0.9207449
11 0.96325356
8 0.53348887
11 0.9641408
11 0.9732132
11 0.9386387
11 0.95241094
11 0.9949753
11 0.9744694
11 0.9965888
11 0.99353385
11 0.9911738
11 0.99152464
11 0.99621636
11 0.98807144
11 0.9973464
11 0.9992644
11 0.99999917
11 0.9999995
11 0.99999964
11 0.9999912
11 0.99999416
11 0.9999999
11 0.999984
11 0.999997
11 0.99999976
11 0.9999844
11 0.99989116
11 0.9978537
11 0.98543686
11 0.8210452
11 0.7206123
11 0.99097097
11 0.9589454
11 0.98954797
3 0.5735704
11 0.7095664
5 0.80789226
11 

5 0.9384716
6 0.5127151
6 0.958989
6 0.88424456
6 0.97707486
6 0.87416804
6 0.9999498
6 0.9993717
5 0.99854773
6 0.8894988
3 0.5001814
6 0.9321062
6 0.9521716
6 0.5168129
6 0.80448973
9 0.9974706
3 0.9775599
9 0.99893886
3 0.997623
3 0.96116865
9 0.99494463
9 0.96779704
6 0.9862336
1 0.9595256
6 0.5303078
10 0.93385994
10 0.99271977
10 0.99933594
10 0.9993284
10 0.9998932
10 0.9998344
10 0.9997825
5 0.99968433
5 0.99677426
5 0.99662423
9 0.75619245
6 0.85589683
6 0.9187794
6 0.971161
6 0.93728584
6 0.7932081
6 0.9848535
6 0.99909747
9 0.6873449
6 0.8782076
10 0.9443383
10 0.95584875
9 0.964938
9 0.9380302
5 0.9959714
5 0.97433156
5 0.9959288
9 0.9975011
11 0.9624586
11 0.5443609
11 0.8010412
11 0.58285517
11 0.8373749
11 0.9827188
11 0.9454883
11 0.53104836
11 0.65710634
10 0.89525765
10 0.8718976
2 0.9922367
8 0.9993216
3 0.60720736
9 0.9999385
9 1.0
9 0.99981576
9 0.9999126
9 0.6145392
6 0.8991792
6 0.7969363
10 0.9947818
9 0.70329815
9 0.97117245
9 0.9998938
6 0.83642
9 0.999479
9 0

10 0.9861931
10 0.78927326
10 0.69636524
2 0.79161334
1 0.99992037
10 0.617572
10 0.50688505
1 0.99996424
1 0.9923596
2 0.80818915
10 0.9998753
10 0.85507685
1 0.9831835
1 0.99653244
1 0.9934077
1 0.9905495
2 0.53275985
1 0.9977404
11 0.91808474
11 0.43342587
9 0.93732315
9 0.99966073
5 0.96694607
1 0.9983944
10 0.8022141
10 0.65015537
5 0.6965881
1 0.9891151
1 0.9900086
1 0.99999833
10 0.51829
1 0.99583
1 0.9846619
1 0.9034265
11 0.7609714
1 0.66145957
10 0.6883094
1 0.9920859
1 0.5286135
1 0.9984723
1 0.96137786
11 0.8438585
9 0.99206626
1 0.99994576
1 0.63937116
5 0.8068096
11 0.740688
9 0.9974125
1 0.97155803
1 0.9996376
1 0.9872895
9 0.9907947
1 0.99104625
1 0.99305975
1 0.9979997
1 0.99998844
1 0.9011668
1 0.99409217
11 0.9970341
11 0.75116175
1 0.9994698
11 0.99395716
1 0.9999956
2 0.6657833
11 0.99227697
11 0.7471649
1 0.9999037
1 0.99108523
11 0.9900787
11 0.8312425
1 0.9087183
1 0.55372703
2 0.5970023
1 0.9998677
1 0.99991786
1 0.9535737
11 0.99894005
9 0.98389006
1 0.9999832

5 0.98805594
10 0.9999325
10 0.99996364
10 0.99996555
10 0.9999075
10 0.99983776
10 0.99928313
10 0.9999187
10 0.9999676
10 0.9995788
10 0.9999511
10 0.99983335
10 0.99985373
5 0.7613928
10 0.9996573
10 0.9999887
10 0.99997854
10 0.9998858
10 0.999944
10 0.99994075
10 0.9999516
5 0.99174666
10 0.9993554
10 0.9999877
10 0.999907
10 0.99979466
10 0.9999831
10 0.9997303
10 0.9999976
10 0.9999614
10 0.9999454
10 0.999985
10 0.99999356
10 0.999902
10 0.99996686
10 0.9996631
10 0.9998776
10 0.9997601
10 0.9999126
10 0.99999
10 0.99998
10 0.99940217
10 0.99998844
10 0.9999615
9 0.9988582
10 0.9999125
10 0.97510046
10 0.9999875
10 0.9998847
10 0.99988854
9 0.98747116
10 0.9999404
10 0.99997723
10 0.9999467
10 0.9998926
10 0.99988353
10 0.99985886
10 0.99945456
10 0.9999776
10 0.9999722
10 0.99999356
10 0.99999774
10 0.99994946
10 0.99998283
10 0.99998796
10 0.9998342
10 0.9998338
10 0.9995808
10 0.99998415
10 0.99991584
10 0.9999466
10 0.99999094
10 0.99977916
11 0.93676627
10 0.99980336
10 0.

1 0.98562753
1 0.9678977
1 0.9941407
1 0.9751296
11 0.9656768
11 0.9966312
11 0.9996314
11 0.99992025
11 0.9999342
11 0.9997706
11 0.99994683
11 0.99998736
11 0.9999956
11 0.9999993
11 0.99999905
11 0.9999981
11 0.9999994
11 0.9999542
11 0.9999771
11 0.99998546
11 0.99998116
11 0.9999063
11 0.99945897
11 0.99999094
11 0.9999809
11 0.9999348
11 0.99996483
11 0.99994016
11 0.99999
11 0.99962974
11 0.99993134
11 0.9999887
11 0.99991214
11 0.99998057
11 0.9999918
1 0.99941504
1 0.99801826
1 0.99962497
1 0.9995521
1 0.9999238
1 0.99991536
1 0.9999502
1 0.9999398
1 0.99994826
1 0.99991107
1 0.9999399
1 0.9999634
1 0.99983704
1 0.9999386
1 0.9999304
1 0.99994755
1 0.99992096
1 0.99985266
1 0.99996924
1 0.99994504
1 0.99997807
1 0.99995303
1 0.9248044
9 0.5655619
9 0.85998607
9 0.99843377
9 0.9985372
9 0.9999639
5 0.45772088
9 0.99937505
9 0.9997818
9 0.9998858
9 0.99993443
9 0.9998909
9 0.9999887
9 0.99991786
9 0.9999988
9 0.9999387
9 0.99999475
9 0.9971756
9 0.99951756
9 0.9992111
9 0.992225

...

3 0.99980146
3 0.99991286
3 0.99995005
3 0.9999255
3 0.99991584
3 0.99995005
3 0.999915
3 0.9999145
3 0.9998609
3 0.9999068
3 0.9999021
3 0.9998894
3 0.99984825
3 0.99989283
3 0.99987304
3 0.9999114
3 0.99991155
3 0.99988997
3 0.9998975
3 0.99993765
3 0.9999461
3 0.99993396
3 0.99988115
3 0.9999455
3 0.99990594
3 0.9998821
3 0.99991
3 0.9997919
3 0.9999281
3 0.999938
3 0.9999484
3 0.9999162
3 0.99987745
3 0.99987876
3 0.9999161
3 0.99994516
3 0.9998776
3 0.99993
3 0.9998087
3 0.9999131
3 0.9999132
3 0.9998591
3 0.99985695
3 0.99987733
3 0.99988437
3 0.9999001
3 0.9999281
3 0.99989593
3 0.9999119
3 0.99989974
3 0.99989533
3 0.99988973
3 0.9999013
3 0.99987996
3 0.9999323
3 0.99993706
3 0.9998672
3 0.99993336
3 0.9998585
3 0.9999331
3 0.9999286
3 0.9999757
3 0.99996245
3 0.99995553
3 0.99996173
3 0.9997389
6 0.892275
4 0.8039101
4 0.9396428
6 0.44825462
6 0.75436
6 0.9710594
6 0.6212477
6 0.8892973
6 0.8061655
6 0.90588695
6 0.9436263
6 0.98682076
6 0.99662846
6 0.9965609
6 0.97243845
6 

2 0.730027
9 0.97200644
2 0.9640919
9 0.97167826
9 0.9999956
9 0.65641844
2 0.8557989
9 0.9027004
9 0.9976107
9 0.99836856
9 0.91768056
9 0.97761154
9 0.9971513
9 0.9995621
9 0.55734724
2 0.99613404
2 0.5542182
3 0.9463222
4 0.9782518
4 0.90802586
3 0.9928352
3 0.99927586
3 0.9958131
3 0.9619933
4 0.52560794
4 0.97903043
4 0.9828196
4 0.9739741
3 0.51811975
3 0.94923514
3 0.5925846
3 0.49582204
3 0.73104376
3 0.97255975
3 0.93963647
3 0.97112167
3 0.89915407
3 0.94139534
4 0.50921214
3 0.9432394
3 0.9796626
3 0.92594063
3 0.9770049
3 0.9770379
3 0.8901227
3 0.97859716
3 0.9750866
3 0.94225365
3 0.94460255
4 0.62854344
3 0.9792271
3 0.90730953
3 0.8469166
3 0.9676867
3 0.9379586
3 0.97040147
3 0.8094193
3 0.8605058
3 0.9835893
3 0.9600275
3 0.9806213
3 0.9844796
3 0.8927231
3 0.97516567
3 0.96772635
3 0.984073
3 0.98261017
3 0.92299557
3 0.98236936
3 0.9812108
4 0.57126886
3 0.97194475
3 0.98217565
3 0.9908762
3 0.96742433
3 0.3692633
3 0.8901712
4 0.97069484
4 0.7531803
4 0.9985776
4 0

4 0.7664141
4 0.80576456
4 0.7316321
4 0.89879125
4 0.80463016
4 0.83532465
4 0.6146346
7 0.8279997
7 0.7649321
7 0.6452854
7 0.7503899
7 0.99596244
7 0.999933
7 0.99998665
7 0.99998975
7 0.99997723
7 0.9999602
7 0.9999759
7 0.99997425
7 0.99998546
7 0.9999597
7 0.99995387
7 0.99996746
7 0.99998415
7 0.99996877
7 0.9999498
7 0.99993205
7 0.99986494
7 0.9998129
7 0.9999527
7 0.99996865
7 0.99997807
7 0.99997115
7 0.9999924
7 0.99995315
7 0.99990475
7 0.9999341
7 0.99995565
7 0.9998628
7 0.99969983
7 0.9999609
7 0.9999918
7 0.9996654
7 0.9522862
7 0.9590419
11 0.9997814
11 0.7582345
2 0.9955687
2 0.8667716
2 0.54096776
11 0.71183145
7 0.8069504
7 0.903296
7 0.85926795
7 0.8989844
5 0.79214853
10 0.8338916
11 0.73729694
2 0.6131264
7 0.87718093
11 0.5201359
2 0.35191047
2 0.85301006
10 0.43116054
10 0.7693253
11 0.9436994
11 0.8030774
11 0.94755954
11 0.95360017
11 0.9732334
11 0.993922
11 0.9763926
11 0.6935998
11 0.8578586
11 0.86970687
11 0.5852836
11 0.9648928
11 0.9874148
11 0.706091

9 0.9999964
9 0.9999254
9 0.9999887
9 0.99060005
9 0.9999968
9 0.99972504
9 0.9998636
9 0.9665064
9 0.99999404
9 0.99732375
9 0.96984565
9 0.97729594
9 0.9999877
9 0.9952642
3 0.6166506
3 0.5407808
3 0.82910377
3 0.70939666
3 0.8340925
1 0.9964014
8 0.46091837
8 0.7525603
3 0.71362734
1 0.55679953
8 0.6723967
11 0.58796865
3 0.9384965
8 0.49987915
3 0.65954936
3 0.6542056
9 0.8213887
9 0.5054182
3 0.71596175
11 0.87077093
11 0.8854273
11 0.983703
11 0.9726601
11 0.93988836
11 0.7829231
11 0.8245044
11 0.9197558
11 0.8745712
11 0.48190677
11 0.6794536
11 0.86972207
11 0.9331958
11 0.4692126
11 0.96874654
11 0.9423689
11 0.7951721
11 0.8960284
3 0.77640486
8 0.505956
3 0.7241195
3 0.999746
9 0.90323025
9 0.9993304
9 0.99999976
9 0.99999964
9 0.9999951
9 0.9998865
9 0.9956352
5 0.5620488
5 0.59340715
4 0.9921484
5 0.6628747
5 0.77740526
5 0.9987319
5 0.98292184
5 0.96132565
5 0.98393136
5 0.9564757
9 0.7822406
9 0.55634516
9 0.98615
5 0.99961805
9 0.93995094
9 0.8579097
5 0.91111714
5 0.7

5 0.9999999
5 0.9999999
5 0.9999999
5 0.99999964
5 0.99999976
5 0.99999917
5 0.9999988
5 0.99999905
5 0.9999974
5 0.99999964
5 0.999998
5 0.99999785
5 0.99999857
5 0.9999895
5 0.99923885
4 0.9995664
4 0.8820041
4 0.99269956
4 0.99912244
4 0.99738437
4 0.9955623
4 0.99117893
4 0.9953513
4 0.99940526
4 0.99965656
4 0.99850816
4 0.9988164
4 0.99848455
4 0.9990368
4 0.98460704
7 0.5717081
7 0.8602435
7 0.9990478
7 0.88767797
4 0.83518845
7 0.7774785
7 0.99999654
7 0.99999905
7 0.9999964
7 0.99999726
7 0.99999046
7 0.99998426
7 0.9999871
7 0.9999875
7 0.9999683
7 0.99999225
7 0.9999722
7 0.9998659
7 0.99999654
7 0.999987
7 0.99998844
7 0.99997115
7 0.99998116
7 0.9999914
7 0.9999962
7 0.9999956
7 0.9999578
7 0.99996364
7 0.9999361
7 0.9997224
7 0.9995877
7 0.9998354
7 0.999941
7 0.99988735
7 0.9999318
7 0.9996043
7 0.9998996
7 0.99986684
7 0.99992085
7 0.9996766
7 0.9996619
7 0.99943846
7 0.99977535
7 0.9999547
7 0.9996979
7 0.99967873
7 0.99960953
7 0.99947876
7 0.997552
7 0.9992803
7 0.99